In [6]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import numpy as np

wildfire_df = pd.read_csv("C:/Users/longl/BurnData.csv")
wildfire_df['geometry'] = wildfire_df['geometry'].apply(wkt.loads)
wildfire_gdf = gpd.GeoDataFrame(wildfire_df, geometry='geometry', crs="EPSG:4326")

# Load airport locations
airport_df = pd.read_excel("C:/Users/longl/airports_processed.xlsx")
airport_df = airport_df.dropna(subset=['latitude_deg', 'longitude_deg'])  

# Get wildfire centroids in lat/long
wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid
wildfire_gdf['centroid_lat'] = wildfire_gdf['centroid'].y
wildfire_gdf['centroid_lon'] = wildfire_gdf['centroid'].x

#  Haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2
    return 2 * R * np.arcsin(np.sqrt(a)) * 1000  # return in meters

#  Compute distances to all airports and pick the closest
results = []

for _, fire in wildfire_gdf.iterrows():
    lat1, lon1 = fire['centroid_lat'], fire['centroid_lon']
    distances = haversine(lat1, lon1, airport_df['latitude_deg'], airport_df['longitude_deg'])
    min_idx = np.argmin(distances)
    nearest_airport=airport_df.iloc[min_idx]
    results.append({
        'Fire ID': fire['unique_id'],
        'CLOSEST_AIRPORT_NAME': airport_df.iloc[min_idx]['name'],
        'AIRPORT IDENT': nearest_airport['ident'],
        'REGION NAME': nearest_airport['region_name'],
        'AIRPORT TYPE':nearest_airport['name'],
        'DISTANCE': distances[min_idx]
    })
# the distance will show in meter
#  Create DataFrame with final results
result_df = pd.DataFrame(results)

print(result_df.head())

C:\Users\longl\AppData\Local\Temp\ipykernel_29064\2227574779.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid


      Fire ID               CLOSEST_AIRPORT_NAME AIRPORT IDENT REGION NAME  \
0  2020_76998  The Florida Keys Marathon Airport          KMTH     Florida   
1  2020_76999  The Florida Keys Marathon Airport          KMTH     Florida   
2  2020_77025  The Florida Keys Marathon Airport          KMTH     Florida   
3  2020_77014  The Florida Keys Marathon Airport          KMTH     Florida   
4  2020_77029  The Florida Keys Marathon Airport          KMTH     Florida   

                        AIRPORT TYPE      DISTANCE  
0  The Florida Keys Marathon Airport  5.583769e+06  
1  The Florida Keys Marathon Airport  5.584827e+06  
2  The Florida Keys Marathon Airport  5.406184e+06  
3  The Florida Keys Marathon Airport  5.692759e+06  
4  The Florida Keys Marathon Airport  5.524422e+06  
